In [ ]:
# Load Transaction Data from a CSV File

### DISCLAIMER: 
### FOR ALL TAX-RELATED QUESTIONS AND CONCERNS PLEASE ASK A CERTIFIED ACCOUNTANT

import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick

# What Year are we generating for
TAX_YEAR = 2020
# Set this to the Mortgage Interest Paid for the year
MORTGAGE_INTEREST = float(3446.73)
EST_DEPRECIATION = float(5363)
EST_AMORTIZATION = float(75)

# Name of the CSV file that has all of the property related transaction data from Min
# / the directory where you are running jupyter notebook
PATH_TO_YOUR_TRANSACTIONS = "transactions.csv"

# Name of the output CSV file that we'll create
OUTPUT_CSV = str(TAX_YEAR) + ' Schedule E Income and Loss.csv'

# Set the name of the column where the transactions are categorized
# In Mint this is usually "Category".  I use Mint's category to link 
# transactions to a property and the add a seperate "Label" column
# after exporting the data for a property from Min
CATEGORY = "Label"

# Load the transaction data from the csv into a dataframe
parse_dates = ['Date']
df = pd.read_csv(PATH_TO_YOUR_TRANSACTIONS, parse_dates=parse_dates)
df.set_index(['Date'], inplace=True)
df['Amount'] = df['Amount'].astype(float)

# Get rid of Mint columns that we don't care about
df.drop(['Original Description','Notes'], axis=1, inplace=True)
df.sort_index(inplace=True)
# df.head()

In [ ]:
# Exclude the data for everything except our tax year
df = df[df.index.year == TAX_YEAR]
# df.head()

In [ ]:
# Find all the categories that have both credits and debits and adjust the overall expense
two_cats_df = df
for label in df[CATEGORY].unique():
    if df[df[CATEGORY] == label]['Transaction Type'].nunique() <= 1:
        two_cats_df = two_cats_df[two_cats_df[CATEGORY] != label]
    else:
        print('Found both credits and debits for category: ' + label)


# Convert the credits to negative debits so that these will be viewed as expenses
def find_refunds(row):
    """ Look for transactions that have a
        credit for a typical "spending" label
        Change these to a debit with a negative
        amount.

        This results in the refunds getting subtracted
        from the total spend calculations
    """
    if row['Transaction Type'] == 'credit' and row[CATEGORY] != "Rent":
        print (row[CATEGORY]+' -- Credit on ' + row.name.strftime('%m/%d/%Y') + ' from '+ \
               row.Description + ' for ${:,.2f}'.format(row.Amount))
        row.Amount *= -1
        row['Transaction Type'] = 'debit'

    if row['Transaction Type'] == 'debit' and row[CATEGORY] == "Rent":
        print (row[CATEGORY]+' -- Debit on ' + row.name.strftime('%m/%d/%Y') + ' from '+ \
               row.Description + ' for ${:,.2f}'.format(row.Amount))
        row['Transaction Type'] = 'credit'

    return row


for label in two_cats_df[CATEGORY].unique():
    if label != 'Rent':
        print('\nBelow are the transactions that were "credits" in the expense category: ' + label)
    else:
        print('\nBelow are the transactions that were "debits" in the income category: ' + label)

    df[df[CATEGORY] == label] = df[df[CATEGORY] == label].apply(find_refunds, axis=1)

print('\nThe overall totals in these categories have been adjusted with these credits/debits.')


In [ ]:
# Get the expense and income data that we'll report on our Schedule E

# If any exist, we exclude closing costs from the year over year expense visualization
df = df[df[CATEGORY] != 'Closing']
# We exclude secuirty deposits from the income/expense reporting
df = df[df[CATEGORY] != 'Security']

# We also remove mortgage payments for tax expense calculation
# We may deduct only the interest payments which we need to get from the lender
df = df[df[CATEGORY] != 'Mortgage']

# Clean up expense data
expenses_df = df[df['Transaction Type'] == 'debit'].copy()
# Re-assign some CATEGORIES to the ones that are on the Tax form 
expenses_df.loc[:][expenses_df[CATEGORY] == 'Water'] = 'Utilities'
expenses_df.loc[:][expenses_df[CATEGORY] == 'Garbage'] = 'Utilities'
expenses_df.loc[:][expenses_df[CATEGORY] == 'Phone'] = 'Supplies'

# Cleanup income data
income_df = df[df['Transaction Type'] == 'credit'].copy()
# # We treat any Pet Deposits as Income so we'll just call it "Rent"
income_df.loc[:][income_df[CATEGORY] == 'Pet Deposit'] = 'Rent'


# Group income and expenses
g = expenses_df.groupby([CATEGORY])
expenses = g.sum()

if MORTGAGE_INTEREST:
    expenses.loc['Mortgage Interest'] = [MORTGAGE_INTEREST]

expenses.sort_index(inplace=True)

if EST_DEPRECIATION:
    expenses.loc['Estimated Depreciation'] = [EST_DEPRECIATION]
    
if EST_AMORTIZATION:
    expenses.loc['Estimated Ammortization'] = [EST_AMORTIZATION]

#g = income_df.groupby([CATEGORY, income_df.index.year])
g = income_df.groupby([CATEGORY])
income = g.sum()




# print(expenses_df[expenses_df[CATEGORY] == 'Insurance'])


In [ ]:
# Output the Tax Related Income and Expense Data to a CSV

# Rename our columns to the terms used on IRS Schedule E
income.rename(index={'Rent':'Rents received'},  inplace=True)
income = income.append(expenses)

# mortgage = pd
income.to_csv(OUTPUT_CSV, float_format="%.2f")
OUTPUT_CSV